# 第9章 質問応答

## 9.4 文書検索モデルの実装

### 9.4.4 BPR によるパッセージの埋め込みの計算

#### 準備

In [ ]:
!pip install datasets faiss-cpu torch transformers[ja,torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.9/599.9 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 40.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ..

#### データセットの読み込み

In [ ]:
from datasets import load_dataset

# Hugging Face Hubのllm-book/aio-passagesのリポジトリから
# AI王データセットのパッセージデータを読み込む
passage_dataset = load_dataset("llm-book/aio-passages", split="train")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Generating train split:   0%|          | 0/4288198 [00:00<?, ? examples/s]

Dataset aio-passages downloaded and prepared to /root/.cache/huggingface/datasets/llm-book___aio-passages/default/1.0.0/df6ac685f2e0d063e5f34cf492b588e9ff34fec7b8956098147aff466c2df41d. Subsequent calls will reuse this data.


In [ ]:
# 読み込まれたパッセージデータの形式と事例数を確認する
print(passage_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'text', 'section', 'title'],
    num_rows: 4288198
})


In [ ]:
from pprint import pprint

# 読み込まれたパッセージデータの内容を確認する
pprint(passage_dataset[0])
pprint(passage_dataset[1])

{'id': 1,
 'pageid': 5,
 'revid': 88740876,
 'section': '__LEAD__',
 'text': 'アンパサンド(&, 英語: '
         'ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" '
         'の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" '
         'の合字であることが容易にわかる字形を使用している。',
 'title': 'アンパサンド'}
{'id': 2,
 'pageid': 5,
 'revid': 88740876,
 'section': '語源',
 'text': '英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" '
         'も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" '
         'のように唱えられていた。また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。"&" '
         'はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。結果として、アルファベットの復唱の最後は "X, Y, '
         'Z, and per se and" という形になった。この最後のフレーズが繰り返されるうちに "ampersand" '
         'と訛っていき、この言葉は1837年までには英語の一般的な語法となった。',
 'title': 'アンパサンド'}


#### トークナイザとモデルの準備

Hugging Face Hubから読み込む場合

In [ ]:
from transformers import AutoModel, AutoTokenizer

# Hugging Face Hubにアップロードされた
# BPRのトークナイザとパッセージエンコーダを読み込む
model_name = "llm-book/bert-base-japanese-v3-bpr-passage-aio"
tokenizer = AutoTokenizer.from_pretrained(model_name)
passage_encoder = AutoModel.from_pretrained(model_name)

ディスクに保存されたモデルを読み込む場合

In [ ]:
from google.colab import drive

drive.mount("drive")

Mounted at drive


In [ ]:
!cp -r drive/MyDrive/llm-book/outputs_bpr .

In [ ]:
from transformers import AutoModel, AutoTokenizer

# ディスクに保存されたBPRのトークナイザとパッセージエンコーダを読み込む
model_path = "outputs_bpr/passage_encoder"
tokenizer = AutoTokenizer.from_pretrained(model_path)
passage_encoder = AutoModel.from_pretrained(model_path)

共通の処理

In [ ]:
# 読み込んだモデルをGPUのメモリに移動させる
device = "cuda:0"
passage_encoder = passage_encoder.to(device)

#### モデルによる埋め込みの計算

In [ ]:
import numpy as np
import torch

def embed_passages(titles: list[str], texts: list[str]) -> np.ndarray:
    """BPRのパッセージエンコーダを用いてパッセージの埋め込みを計算"""
    # パッセージにトークナイザを適用
    tokenized_passages = tokenizer(
        titles,
        texts,
        padding=True,
        truncation="only_second",
        max_length=256,
        return_tensors="pt",
    ).to(device)

    # トークナイズされたパッセージを実数埋め込みに変換
    with torch.inference_mode():
        with torch.cuda.amp.autocast():
            encoded_passages = passage_encoder(
                **tokenized_passages
            ).last_hidden_state[:, 0]

    # 実数埋め込みをNumPyのarrayに変換
    emb = encoded_passages.cpu().numpy()
    # 0未満の値を0に、0以上の値を1に変換
    emb = np.where(emb < 0, 0, 1).astype(bool)
    # bool型からuint8型に変換
    emb = np.packbits(emb).reshape(emb.shape[0], -1)
    return emb

In [ ]:
# パッセージデータのすべての事例に埋め込みを付与する
passage_dataset = passage_dataset.map(
    lambda examples: {
        "embeddings": list(
            embed_passages(examples["title"], examples["text"])
        )
    },
    batched=True,
)

Map:   0%|          | 0/4288198 [00:00<?, ? examples/s]

In [ ]:
# 埋め込みを適用したパッセージデータの形式と事例数を確認する
print(passage_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'text', 'section', 'title', 'embeddings'],
    num_rows: 4288198
})


In [ ]:
# 埋め込みを適用したパッセージデータの内容を確認する
pprint(passage_dataset[0])

{'embeddings': [133,
                162,
                145,
                21,
                151,
                215,
                254,
                119,
                214,
                80,
                4,
                189,
                177,
                53,
                100,
                115,
                68,
                177,
                70,
                103,
                41,
                90,
                127,
                227,
                113,
                27,
                71,
                148,
                92,
                162,
                176,
                133,
                105,
                99,
                16,
                16,
                52,
                70,
                132,
                46,
                2,
                32,
                211,
                149,
                29,
                103,
                53,
                233,
               

In [ ]:
# 埋め込みを適用したパッセージデータをディスクに保存する
passage_dataset.save_to_disk("outputs_bpr/embedded_passages")

Saving the dataset (0/7 shards):   0%|          | 0/4288198 [00:00<?, ? examples/s]

#### Google ドライブへの保存

In [ ]:
from google.colab import drive

drive.mount("drive")

In [ ]:
# 保存されたパッセージデータをGoogleドライブのフォルダにコピーする
!cp -r outputs_bpr/embedded_passages drive/MyDrive/llm-book/outputs_bpr